In [9]:
import tensorflow as tf
import argparse
import os.path
from cifar10_models import *
from cifar10_input import *
from utils import *

tf.logging.set_verbosity(tf.logging.INFO)

#### SET THIS BEFORE RUNNING
NAME = "cifar10/cifar10_scm_001"
try:
    os.makedirs(CHECKPOINT.format(NAME))
except FileExistsError:
    pass

tf.reset_default_graph()
train_files, test_files = maybe_download_and_extract()
train_data = Cifar10Record(train_files, epochs=1)
test_data = Cifar10Record(test_files, epochs=1)

model = Cifar10ShallowConvolutionalModel(test_data.image, test_data.label)
model = Cifar10ShallowConvolutionalModel(test_data.image, test_data.label)

try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(NAME),x[:-5]) for x in os.listdir(ROOT.format(NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    checkpoints = []
model.saver.recover_last_checkpoints(checkpoints)
try: latest_checkpoint = model.saver.last_checkpoints[-2]
except: latest_checkpoint = None

In [10]:
#### Rows are true label, columns are the prediction
def p_confusion_matrix(model, checkpoint=None):
    cf = np.zeros((10,10))
    with tf.Session() as sess:
        model._restore_or_init(sess, checkpoint)
        try:
            while True:
                probas, y_ = sess.run([model.proba, model.y_])
                index = np.argmax(y_, axis=1)
                for i in range(len(index)):
                    cf[index[i]] += probas[i]
        except tf.errors.OutOfRangeError as e:
            pass
    return cf

In [21]:
print(ptcfs[-4] - ptcfs[-5])

[[  9.98974888e-04  -9.48268765e-04   6.18551270e-04  -4.36208432e-04
    4.29158727e-04  -1.80529061e-05  -2.31504173e-04  -1.48512577e-04
   -2.43370071e-04  -2.07679600e-05]
 [  6.48479830e-04  -4.04241475e-03   2.66310162e-04  -1.52239237e-05
    8.99534779e-05   1.47145323e-04   7.69653708e-05   2.12478603e-05
    9.41699211e-04   1.86583744e-03]
 [  9.98592830e-05  -2.54461761e-04   3.72250720e-03  -5.84729422e-04
   -3.62172866e-04  -3.86986283e-04  -1.03511002e-03  -1.07122466e-03
   -1.39495938e-04   1.18144651e-05]
 [  3.21836981e-04  -3.72742895e-04   1.62066491e-03  -1.36796861e-04
   -2.84547807e-04   7.21530034e-05  -6.52775764e-04  -7.48019139e-04
    1.24274599e-04   5.59529680e-05]
 [  1.78330985e-04  -1.89725791e-04   1.76350403e-03   1.22376383e-04
   -2.18010816e-04  -6.35250939e-05  -3.33936986e-04  -1.21847992e-03
   -6.22427880e-05   2.17099996e-05]
 [  2.33945396e-04  -3.05155224e-04   1.50956818e-03  -3.29376452e-05
   -1.88762399e-04   2.64001232e-06  -3.49781

In [ ]:
import os
pickled = "cifar10_test_cm.npy"
try:
    cfs = np.load(pickled)
    cfs = list(cfs)
except Exception as e:
    print(e)
    cfs = []
for i in range(len(cfs), len(checkpoints)):
    checkpoint = checkpoints[i]
    cfs.append(p_confusion_matrix(model, checkpoint))
    if not i%10:
        print(i/len(checkpoints))
        np.save(pickled, cfs)


In [5]:
cfs

[array([[ 388.78974461,   59.91234349,   86.76859866,   46.79960131,
           60.1933101 ,   47.80819764,   32.24641226,   44.57667073,
          145.87462281,   87.03049528],
        [  72.87848678,  300.98680222,   38.09582287,   82.24210461,
           69.20837844,   53.10888146,   35.95682241,   63.92252891,
           98.10990451,  185.49026779],
        [  84.70911226,   29.95115963,  201.63630298,  117.12765985,
          192.75986692,  118.87914528,   91.21517072,   86.36360284,
           36.07177969,   41.28619838],
        [  48.51846761,   51.06207046,   86.03314288,  221.59453386,
          126.89795233,  181.9653959 ,   91.92801605,  103.49782132,
           33.02883112,   55.47376697],
        [  44.95463241,   30.72686578,  106.42218388,  111.54069873,
          291.86307346,  109.93905355,  115.58134264,  126.79775742,
           25.86045628,   36.3139362 ],
        [  35.06143239,   31.47027529,   90.33534694,  196.46215694,
          143.42998681,  248.87309011,   

In [3]:
import numpy as np
ptcfs = np.load("cifar10_test_cm.npy")
ptrcfs = np.load("cifar10_train_cm.npy")

In [4]:
#### TODO: may want to use normalization as in Feinberg 1970 An iterative procedure for estimation in contingency tables
for i in range(len(ptcfs)):
    for row in range(len(ptcfs[i])):
        ptcfs[i][row] = ptcfs[i][row]/np.sum(ptcfs[i][row])
        ptrcfs[i][row] = ptrcfs[i][row]/np.sum(ptrcfs[i][row])

In [23]:
x = range(len(ptcfs))
print(len(ptcfs))

711


In [25]:
import scipy.signal
import matplotlib.pyplot as plt
filt = scipy.signal.gaussian(10,3)
filt /= sum(filt)
fig, axarr = plt.subplots(10,10)
for r in range(10):
    for c in range(10):
        axarr[r,c].set_xticklabels([])
        axarr[r,c].set_yticklabels([])
        axarr[r,c].plot(x[20:690], scipy.signal.convolve(ptcfs[:,r,c], filt,mode='same')[20:690],'b')
        axarr[r,c].plot(x[20:690], scipy.signal.convolve(ptrcfs[:,r,c], filt,mode='same')[20:690],'r')
fig.set_size_inches(20, 20)
#plt.show()
fig.savefig('confusion_matrix_by_time_cifar10.png')